In [71]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from concurrent.futures import ThreadPoolExecutor


def get_details_of_property(property_url):
    """ Function to get all the characteristics of an individual property from the property url """
    property_details = requests.get(property_url)
    soup = BeautifulSoup(property_details.text,"html.parser")
    #print(soup.prettify())
    details = []
    for detail in soup.find_all("script"):
        details.append(detail.text)
    data = details[1]
    # cleaning the data and converting it to a dictionary
    data = data.replace("\n", "")
    data1 = data.strip(" window.dataLayer = [            ];")
    data1 = data1.replace(" ","")
    property_data = json.loads(data1)
    flat_property_data = dict_flatten(property_data)
    return(flat_property_data)


def dict_flatten(in_dict, dict_out=None, parent_key=None, separator="_"):
    """ Function to convert nested dictionary to a single dictionary """
    if dict_out is None:
        dict_out = {}
    for k, v in in_dict.items():
        if parent_key :
            k = f"{parent_key}{separator}{k}"
        else:
            k = k
        if isinstance(v, dict):
            dict_flatten(in_dict=v, dict_out=dict_out, parent_key=k)
            continue
        dict_out[k] = v
    return(dict_out)


def get_list_of_property():
    """ Function to list of property urls from the webpage """
    property_url_list=[]
    categories = ['apartment', 'house']
    for category in categories:
        #print(category)
        options = Options()
        options.add_argument("--headless")
        url = f"https://www.immoweb.be/en/search/{category}/for-sale?countries=BE"
        driver =webdriver.Firefox(options = options) # without opening the browser
        driver.get(url)
        driver.implicitly_wait(10)  # implicitly waiting for the cookie window to pop up
        cookie_button = driver.find_element(By.XPATH, """//*[@id="uc-btn-accept-banner"]""")
        cookie_button.click()
        pages = driver.find_elements(By.XPATH, """/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[3]/div/div[1]/div[1]/div[1]/div/div[1]/div/nav/ul/li[4]/a/span[2]""")
        for page in pages:
            totalpages = page.text
        driver.close()
        #print(totalpages)
        totalpages = 10 # hardcoding for checking the loop
        for i in range(int(totalpages)):
            page_url = f"https://www.immoweb.be/en/search/apartment/for-sale?countries=BE&page={i+1}&orderBy=relevance"
            option = Options()
            option.add_argument("--headless")
            driver = webdriver.Firefox(options = option) #without opening the browser
            driver.get(page_url)
            driver.implicitly_wait(10)  # implicitly waiting for the cookie window to pop up
            cookie_button = driver.find_element(By.XPATH, """//*[@id="uc-btn-accept-banner"]""")
            cookie_button.click()
            property_list = driver.find_elements(By.XPATH, """//*[@class="card__title-link"]""")
            for title in property_list:
                property_url_list.append(title.get_attribute("href"))
            driver.close()
    print(len(property_url_list))
    return(property_url_list)
    

In [72]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

def get_all_property_data():
    """ Function to collect the details of each property from the list of urls """
    all_property_data_list = []
    property_url_list = get_list_of_property()
    pool = ThreadPoolExecutor(max_workers = 8)
    results = pool.map(get_details_of_property, property_url_list)
    for res in results:
        all_property_data_list.append(res)
    pool.shutdown()
    print(len(all_property_data_list))
    df = pd.DataFrame.from_dict(all_property_data_list)
    #print(df)
    df.to_csv("propertydata.csv")

get_all_property_data()

333
333
760
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y
done
y